In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
faa_df = pd.read_csv("faa_ai_prelim.csv")

# View first 5 rows
# print("First 5 rows:")
# display(faa_df.head())

# # Dataset info
# print("\nDataset Info:")
# print(faa_df.info())

# # Missing value count
# print("\nMissing Values:")
# print(faa_df.isnull().sum())


### onverting date and time columns to a single datetime object


In [7]:
faa_df['EVENT_DATETIME'] = pd.to_datetime(
    faa_df['EVENT_LCL_DATE'] + ' ' + faa_df['EVENT_LCL_TIME'],
    errors='coerce',
    format='%d-%b-%y %H:%M:%SZ'
)

# display(faa_df[['EVENT_LCL_DATE', 'EVENT_LCL_TIME', 'EVENT_DATETIME']].head())


### Extract the attributes and do taskes

In [8]:

required_columns = [
    'ACFT_MAKE_NAME',     
    'LOC_STATE_NAME',     
    'ACFT_MODEL_NAME',    
    'RMK_TEXT',           
    'FLT_PHASE',          
    'EVENT_TYPE_DESC',    
    'FATAL_FLAG'          
]

faa_selected = faa_df[required_columns].copy()

# display(faa_selected.head())


#### Replacing all Fatal Flag missing values with no

In [9]:
# Replace missing values in FATAL_FLAG with 'No'
faa_selected['FATAL_FLAG'] = faa_selected['FATAL_FLAG'].fillna('No')

# Verify replacement
print("Missing values in Fatal Flag after replacement:", faa_selected['FATAL_FLAG'].isnull().sum())
# display(faa_selected.head())


Missing values in Fatal Flag after replacement: 0


#### Fill missing FLT_PHASE and ACFT_DMG_DESC using the mode 

In [10]:

faa_df['FLT_PHASE'].fillna(faa_df['FLT_PHASE'].mode()[0], inplace=True)

faa_df['ACFT_DMG_DESC'].fillna(faa_df['ACFT_DMG_DESC'].mode()[0], inplace=True)

# Verify missing values replaced
print("Missing values after filling with mode:")
print(faa_df[['FLT_PHASE', 'ACFT_DMG_DESC']].isnull().sum())

# Check the number of observations
# print("\nNumber of observations before cleaning:", len(faa_df))


Missing values after filling with mode:
FLT_PHASE        0
ACFT_DMG_DESC    0
dtype: int64


C:\Users\Admin\AppData\Local\Temp\ipykernel_13220\1643660826.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  faa_df['FLT_PHASE'].fillna(faa_df['FLT_PHASE'].mode()[0], inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13220\1643660826.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

### Remove observations where aircraft names are not available , columns with more than 75 missing values and checking 

In [11]:

faa_df = faa_df.dropna(subset=['ACFT_MAKE_NAME'])

faa_df = faa_df.dropna(axis=1, thresh=len(faa_df) - 75)

# print("Number of observations after cleaning:", len(faa_df))


### Group the dataset by aircraft name and view counts

In [12]:

aircraft_counts = faa_df.groupby('ACFT_MAKE_NAME').size().reset_index(name='Count')

print("Number of times each aircraft type appears in the dataset:")
# display(aircraft_counts)


Number of times each aircraft type appears in the dataset:


### Display observations where Fatal Flag is "Yes"

In [13]:
fatal_yes = faa_df[faa_df['FATAL_FLAG'].str.strip().str.upper() == 'YES']
print("Observations with Fatal Flag = 'Yes':")
# display(fatal_yes)


Observations with Fatal Flag = 'Yes':


### Performing One-hot encode ACFT_DMG_DESC column and merging encoded columns in to dataframe

In [14]:
acft_dmg_encoded = pd.get_dummies(faa_df['ACFT_DMG_DESC'], prefix='DMG', drop_first=True)

faa_df = pd.concat([faa_df, acft_dmg_encoded], axis=1)

print("Columns after one-hot encoding:")
print(faa_df.columns.tolist())
# faa_df.head(5)


Columns after one-hot encoding:
['UPDATED', 'ENTRY_DATE', 'EVENT_LCL_DATE', 'EVENT_LCL_TIME', 'LOC_CITY_NAME', 'LOC_STATE_NAME', 'RMK_TEXT', 'EVENT_TYPE_DESC', 'FSDO_DESC', 'REGIST_NBR', 'FLT_NBR', 'ACFT_OPRTR', 'ACFT_MAKE_NAME', 'ACFT_MODEL_NAME', 'ACFT_DMG_DESC', 'FLT_ACTIVITY', 'FLT_PHASE', 'MAX_INJ_LVL', 'FATAL_FLAG', 'FLT_CRW_INJ_NONE', 'FLT_CRW_INJ_MINOR', 'FLT_CRW_INJ_FATAL', 'PAX_INJ_NONE', 'PAX_INJ_SERIOUS', 'EVENT_DATETIME', 'DMG_Minor', 'DMG_Substantial', 'DMG_Unknown']


In [15]:
# faa_df['DMG_Substantial']=faa_df['DMG_Substantial'].map({True:1,False:0})
# faa_df['DMG_Unknown']=faa_df['DMG_Unknown'].map({True:1,False:0})

faa_df['DMG_Substantial'] = faa_df['DMG_Substantial'].astype(int)
faa_df['DMG_Unknown'] = faa_df['DMG_Unknown'].astype(int)
faa_df['DMG_Minor'] = faa_df['DMG_Minor'].astype(int)

In [16]:
faa_df.head(2)

,UPDATED,ENTRY_DATE,EVENT_LCL_DATE,EVENT_LCL_TIME,LOC_CITY_NAME,LOC_STATE_NAME,RMK_TEXT,EVENT_TYPE_DESC,FSDO_DESC,REGIST_NBR,...,FATAL_FLAG,FLT_CRW_INJ_NONE,FLT_CRW_INJ_MINOR,FLT_CRW_INJ_FATAL,PAX_INJ_NONE,PAX_INJ_SERIOUS,EVENT_DATETIME,DMG_Minor,DMG_Substantial,DMG_Unknown
0,No,19-FEB-16,19-FEB-16,00:45:00Z,MARSHVILLE,North Carolina,"AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...",Accident,FAA Charlotte FSDO-68,N61WB,...,Yes,NaN,NaN,1.0,NaN,NaN,2016-02-19 00:45:00,0,0,1
1,No,19-FEB-16,18-FEB-16,23:55:00Z,TAVERNIER,Florida,AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...,Incident,FAA Miami FSDO-19,N705RP,...,NaN,1.0,NaN,NaN,NaN,NaN,2016-02-18 23:55:00,0,0,1


# reconstion


In [17]:
def pca_reconstruction_error(X, ks):
    """
    Compute PCA reconstruction error using Eigenfaces method
    (Turk & Pentland 1991).
    """
    # Keep only numeric features
    # X = X.select_dtypes(include=[np.number]).copy()
    # X = X.fillna(X.mean())   # handle NaN
    
    # Step 1: Mean center
    mu = X.mean(axis=0)
    A = X - mu
    
    # Step 2: Compute A^T A
    ATA = A.T @ A
    
    # Step 3: Eigen decomposition
    eigvals, eigvecs = np.linalg.eigh(ATA)
    
    # Sort eigenvalues descending
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]
    
    errors = []
    for k in ks:
        # Step 4: Take top-k eigenvectors
        Ek = eigvecs[:, :k]
        
        # Step 5: Project & reconstruct
        W = A @ Ek         # projections
        L = W @ Ek.T       # linear combinations
        R = L + mu         # reconstructed points
        
        # Step 6: Compute error
        err = np.sum((X - R)**2)
        errors.append( err)
    return errors    
    
    # return pd.DataFrame(errors, columns=["k", "reconstruction_error"])


In [19]:


ks = [1,2,3,5,7,10,12]   # up to available features
error = pca_reconstruction_error(faa_df, ks)
# print("Housing Reconstruction Errors:\n", house_res)

plt.plot(ks, error, marker="o")
plt.title("PCA Reconstruction Error – Housing")
plt.xlabel("Number of Components (k)")
plt.ylabel("Error")
plt.grid(True)
plt.show()


TypeError: can only concatenate str (not "int") to str

# Advance tasks

In [ ]:
# Define keywords for flight phases
phase_keywords = {
    'LANDING': 'LANDING',
    'TAKEOFF': 'TAKEOFF',
    'CRUISE': 'CRUISE',
    'APPROACH': 'APPROACH'
}

# Function to extract flight phase from RMK_TEXT
def extract_flight_phase(text):
    if pd.isnull(text):
        return 'UNKNOWN'
    text_upper = text.upper()
    for keyword, phase in phase_keywords.items():
        if keyword in text_upper:
            return phase
    return 'UNKNOWN'

# Apply extraction to create FLIGHT_PHASE_TEXT
faa_df['FLIGHT_PHASE_TEXT'] = faa_df['RMK_TEXT'].apply(extract_flight_phase)

# Compare with existing FLT_PHASE
print("Comparison of FLT_PHASE vs FLIGHT_PHASE_TEXT:")
display(faa_df[['FLT_PHASE', 'FLIGHT_PHASE_TEXT']].head())
